In [1]:
import findspark
import string
#findspark.init("/home/ec2-user/spark-2.4.5-bin-hadoop2.7/")
findspark.init("/usr/local/spark/spark-2.4.0-bin-hadoop2.7/")
import pyspark
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, IntegerType
from pyspark.sql.functions import udf
from pyspark.ml.feature import CountVectorizerModel, IDFModel, StandardScalerModel, Tokenizer
from pyspark.ml.classification import LogisticRegressionModel


from urllib.parse import unquote

def to_ngram(payload_obj):
    n=2
    payload = str(payload_obj)
    ngrams = ''
    for i in range(0,len(payload)-n + 1):
        ngrams += payload[i:i+n]+ ' '
    return ngrams[:-1]


# define a function to compute sentiments of the received tweets
def get_prediction(queries):
    try:
        queries = queries.map(lambda w: Row(payload=w))
        queries = sqlc.createDataFrame(queries)
        queries = queries.withColumn('ngrams', ngrams(queries['payload']))
        queries = tokenizer.transform(queries)
        queries = vectorizer.transform(queries)
        queries = idf_model.transform(queries)
        queries = scalerModel.transform(queries)
        preds = model.transform(queries)
        preds.select('payload','prediction').show(truncate=False)

    except : 
        print('No data')
        

APP_NAME = "BigData"
conf = pyspark.SparkConf().setAll([ ('spark.app.name', APP_NAME),('spark.executor.memory', '8g'), ('spark.cores.max', '2'), ('spark.driver.memory','8g')])
sc = SparkContext(conf=conf)
ssc = StreamingContext(sc, batchDuration= 3)
sqlc = SQLContext(sc)
    
ngrams = udf(to_ngram, StringType())
tokenizer = Tokenizer.load('../models/Tokenizer')
vectorizer = CountVectorizerModel.load('../models/Vectorizer')
idf_model = IDFModel.load('../models/idf')
scalerModel = StandardScalerModel.load('../models/scalerModel')
model = LogisticRegressionModel.load('../models/Logistic_Regression_Model')

# lines = ssc.textFileStream("/home/ec2-user/Notebooks/BigData-Http-injections/data")
lines = ssc.textFileStream("../data")
lines.foreachRDD(get_prediction)
ssc.start()             
ssc.awaitTermination()


No data
No data
+--------------------------------------+----------+
|payload                               |prediction|
+--------------------------------------+----------+
|/tagarela 1/                          |0.0       |
|/realplayer10/                        |0.0       |
|/6404573/                             |0.0       |
|/000023953/                           |0.0       |
|/_mocks/                              |0.0       |
|/javascript/oaerrordetailpage.svn-base|0.0       |
|/041440/                              |0.0       |
|/mgnews/                              |0.0       |
|/123004/                              |0.0       |
|/nettoys/                             |0.0       |
|/flag_japan/                          |0.0       |
|/666865/                              |0.0       |
|/136421/                              |0.0       |
|/javascript/todo.trace                |0.0       |
|/directory_com-coo/                   |0.0       |
|/seagate pro firewall  an crack/      |0.0     

KeyboardInterrupt: 